In [1]:
!nvidia-smi

Thu Jul 13 21:58:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 T...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8               10W /  N/A|    159MiB /  4096MiB |     30%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!git clone https://github.com/HusseinMansourMohd/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


C:/Users/hussein/Adapter_for_small_medical_datasets_classification/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification-1/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/.git: Filename too long
Cloning into 'Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification'...


In [3]:
import os
print(os.listdir())

['.git', 'colab.ipynb', 'MedVit_adapter.ipynb', 'MedVit_adapter.py', 'README.md', 'utils.py']


In [5]:
!pip install summary

'summary' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
from torchsummary import summary

ModuleNotFoundError: No module named 'torchsummary'

In [ ]:
!pip install timm
!pip install einops

In [6]:
import os
print(os.listdir())

['.git', 'colab.ipynb', 'MedVit_adapter.ipynb', 'MedVit_adapter.py', 'README.md', 'utils.py']


In [7]:
import MedVit_adapter
print(dir(MedVit_adapter))

ModuleNotFoundError: No module named 'einops'

In [8]:
from MedVit_adapter import MedViT_small as small

ModuleNotFoundError: No module named 'einops'

In [9]:
model = small()

NameError: name 'small' is not defined

In [10]:
model.proj_head[0]

NameError: name 'model' is not defined

In [11]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=1, bias = True)


NameError: name 'model' is not defined

In [12]:
!pip install medmnist

In [13]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [ ]:
data_flag = 'breastmnist'
# [tissuemnist , pathmnist, chestmnist, dermamnist, octmnisr]
# ,pnemonismnist , retinamnist, bloodmnist, tissuemnist, organcmist, organs ]

download = True

NUM_EPOCHS = 10
BATCH_SIZE = 10
LR = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist,info['python_class'])

In [15]:
from torchvision.transforms.transforms import Resize
#preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])
test_transform =  transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)


# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

In [ ]:
print(train_dataset)
print("==============")
print(test_dataset)

In [ ]:
# defune loss function and optimizer
# define loss function and optimizer
lr = 0.001
if task == "binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = model.to(device)  # Move model to the specified device

In [16]:
#train
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d%d]'%(epoch+1, NUM_EPOCHS))
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda() , targets.cuda()
        #forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.floar32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [ ]:
# evaluation
from sklearn.metrics import roc_auc_score, accuracy_score

def test(split):
    model.eval()
    y_true = torch.tensor([]).cuda()
    y_score = torch.tensor([]).cuda()

    data_loader = train_loader_at_eval if split =='train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)  # here is where the model should produce outputs

            if task == 'multi-task, binary-class':
                targets = targets.to(torch.float32)
                loss = criterion(outputs, targets)
                print('loss=',loss)
            else:
                targets = targets.to(torch.long)  # targets for classification tasks are long integers
                targets = targets.float()
                #outputs = outputs.softmax(dim=1)
                #print(outputs)

                loss = criterion(outputs, targets)
                #print('loss=',loss)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs.softmax(dim=0)), 0)  # apply softmax here for evaluation
            print(y_score)
        y_true = y_true.cpu().numpy()
        
        
        y_score = y_score.detach().cpu().numpy()

        y_pred = (y_score > 0.5).astype(int)

        print("y_score.shape:",y_score.flatten().shape)
        print("y_true.shape:",y_true.flatten().shape)

        acc = accuracy_score(y_true, y_pred)
        print(acc)
        print(y_true[1:10])
        print(y_score[1:10])
        auc = roc_auc_score(y_true, y_score, multi_class='ovr')
        #evaluator = Evaluator(data_flag, split)
        #metrics = evaluator.evaluate(y_score)


        print('%s auc: %.3f acc: %.3f' % (split,[acc, auc]))

print('==> Evaluating...')
test('train')
test('test')